# Reaction–diffusion · Front-speed walkthrough

A compact, in-notebook reproduction of the Fisher–KPP front-speed
analysis implemented in `src/reaction_diffusion/rd_front_speed_experiment.py`.
We drive the simulator with a small grid, summarize the empirical speed,
and outline where persistent figures/logs would land.

In [ ]:
import sys
from pathlib import Path

def _ensure_repo_on_path():
    here = Path.cwd().resolve()
    for candidate in (here, *list(here.parents)[:4]):
        utils_py = candidate / 'notebooks' / 'utils.py'
        if utils_py.exists():
            path_str = str(candidate)
            if path_str not in sys.path:
                sys.path.insert(0, path_str)
            return candidate
    return here

_ensure_repo_on_path()


In [ ]:
from notebooks.utils import ensure_repo_path, allocate_artifacts, preview_json

repo_root = ensure_repo_path()
print(f"Using repo root: {repo_root}")

In [ ]:
try:
    from src.reaction_diffusion.rd_front_speed_experiment import run_sim, plot_and_save
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from src.reaction_diffusion.rd_front_speed_experiment import run_sim, plot_and_save

import numpy as np

params = {
    "N": 256,
    "L": 120.0,
    "D": 1.0,
    "r": 0.25,
    "T": 50.0,
    "cfl": 0.2,
    "seed": 7,
    "level": 0.1,
    "x0": -40.0,
    "fit_frac": (0.55, 0.9),
    "noise_amp": 0.0,
}
data = run_sim(**params)
summary = {
    "c_meas": float(data["c_meas"]),
    "c_abs": float(data["c_abs"]),
    "c_th": float(data["c_th"]),
    "rel_err": float(data["rel_err"]),
    "r2": float(data["r2"]),
    "dx": float(data["dx"]),
    "dt": float(data["dt"]),
    "steps": int(data["steps"]),
}
print(preview_json(summary))


In [ ]:
from IPython.display import Image, display

artifacts = allocate_artifacts("reaction_diffusion", "front_speed_notebook_demo")
plot_and_save(data, artifacts["figure"])
print(preview_json({name: str(path) for name, path in artifacts.items()}))
display(Image(filename=str(artifacts["figure"])))


## Notebook checklist

* Theoretical baseline: $c_{\text{th}} = 2\sqrt{Dr}$.
* Empirical fit uses robust regression on the tracked interface.
* Re-run with higher `N`/`T` for production validation.
* Full CLI, with logging and figures persisted, lives in:
  ```bash
  python -m src.reaction_diffusion.rd_front_speed_experiment --help
  ```